In [ ]:
import os, re
import nltk
# nltk.download('stopwords')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from tqdm import tqdm

lem = WordNetLemmatizer()

def text_preprocess(a_string):
    """
    clean the text: remove punctuations; lemmatization
    """
    a_string = a_string[:int(len(a_string) * 0.5)]
    main_words = re.sub('[^a-zA-Z]', ' ', a_string)  # Retain only alphabets
    main_words = (main_words.lower()).split()
    main_words = [w for w in main_words if not w in set(stopwords.words('english'))]  # Remove stopwords
    main_words = [lem.lemmatize(w) for w in main_words if len(w) > 1]  # Group different forms of the same word
    main_words = ' '.join(main_words)
    return main_words


## 0. Preprocessing
* clean the job_description column
* combine individual files into one

In [ ]:
RAW_TEXT_COLNAME = 'lnks_job_description'
CLEANED_TEXT_COLNAME = 'clean_job_description'

In [ ]:
labeled_filename = 'labelled_data/all_labelled_jobs.csv' # outout filename
labeled_files = [os.path.join('labelled_data', item) for item in os.listdir('labelled_data') if '.csv' in item] # individual files to be processed

filtered_df = None
for file in tqdm(labeled_files, total=len(labeled_files)):
    job_df = pd.read_csv(file)  # required columns: lnks_job_title, lnks_job_description
    if filtered_df is None:
        filtered_df = job_df
    else:
        filtered_df = pd.concat([filtered_df, job_df])

tqdm.pandas()
filtered_df[CLEANED_TEXT_COLNAME] = filtered_df[RAW_TEXT_COLNAME].progress_apply(text_preprocess)
filtered_df.to_csv(labeled_filename, index=False)

### Need to run this for both the labeled data and the raw unlabeled data